In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("Invoice_20201220.csv", sep="|", encoding="latin-1")
df

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,19767_GEO (0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,19767_GEO (0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,19767_GEO (0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,19767_GEO (0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


In [2]:
# 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".
df['SITE_NAME'] = df['SITE_NAME'].str[11:17]

def clean_site_name(site_name):
    result = re.search("\d{4}[a-zA-Z]{2}", site_name)
    if result is None:
        return np.NaN
    else:
        return site_name
    return result

df["SITE_NAME"] = [clean_site_name(site_name) for site_name in df["SITE_NAME"]]
df

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,0632KO,FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,055100168,01/07/2020,30/09/2020,2 230,"294,07"


In [3]:
print(df.isnull().sum())
print(df[df['SITE_NAME'].isnull()])

SITE_NAME       11
INVOICE_NAME     0
BEGIN            0
END              0
KWH_PERIOD       0
PERIOD_EUR       0
dtype: int64
      SITE_NAME INVOICE_NAME       BEGIN         END KWH_PERIOD PERIOD_EUR
35919       NaN   1300082549  01/01/2018  31/01/2018      1 628     214,02
35920       NaN   1300082549  01/02/2018  28/02/2018      1 459     192,56
35921       NaN   1300082549  01/03/2018  31/03/2018      1 633     215,18
35922       NaN   1300082549  01/04/2018  30/04/2018      1 765     232,14
35923       NaN   1300082549  01/05/2018  31/05/2018      1 651     217,07
35924       NaN   1300082549  01/06/2018  30/06/2018      1 976     257,52
35925       NaN   1300082549  01/07/2018  31/07/2018      2 257     292,29
35926       NaN   1300082549  01/08/2018  31/08/2018      2 236     288,70
35927       NaN   1300082549  01/09/2018  30/09/2018        413      60,83
35928       NaN   1300082549  01/10/2018  31/10/2018      1 267     168,65
35929       NaN   1311297011  01/11/2018  28/11/

In [4]:
# 2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.
df = df.drop(columns=["INVOICE_NAME"])
df

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,01/07/2020,30/09/2020,2 230,"294,07"


In [5]:
# 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.
df["BEGIN"] = pd.to_datetime(df["BEGIN"],format='%d/%m/%Y',errors='coerce')
df["END"] = pd.to_datetime(df["END"],format='%d/%m/%Y',errors='coerce')
df["DAYS_IN_PERIOD"] = df["END"].sub(df["BEGIN"], axis=0).dt.days

# Je crée aussi une colonne bis avec la date de fin incluse dans le nb de jours
df["DAYS_IN_PERIOD_bis"] = pd.Series((df["END"].sub(df["BEGIN"], axis=0).dt.days)+1)

df

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_IN_PERIOD,DAYS_IN_PERIOD_bis
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",30,31
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",27,28
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",30,31
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",29,30
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",30,31
...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3 028,"397,18",123,124
51428,0632KO,2020-01-01,2020-03-31,2 163,"283,57",90,91
51429,0632KO,2020-04-01,2020-06-30,2 160,"312,55",90,91
51430,0632KO,2020-07-01,2020-09-30,2 230,"294,07",91,92


In [6]:
# 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.
df["KWH_PERIOD"] = df["KWH_PERIOD"].str.replace(" ", "")
df["KWH_PERIOD"] = pd.to_numeric(df["KWH_PERIOD"], errors='coerce')

df["AVG_DAILY_CONSUMPTION"] = df.apply(lambda x: "NaN" if x["DAYS_IN_PERIOD"] <= 0 else x["KWH_PERIOD"]/x["DAYS_IN_PERIOD"], axis=1)
df["AVG_DAILY_CONSUMPTION"] = pd.to_numeric(df["AVG_DAILY_CONSUMPTION"], errors='coerce')

df["AVG_DAILY_CONSUMPTION_bis"] = df.apply(lambda x: "NaN" if x["DAYS_IN_PERIOD_bis"] <= 0 else x["KWH_PERIOD"]/x["DAYS_IN_PERIOD_bis"], axis=1)

df

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_IN_PERIOD,DAYS_IN_PERIOD_bis,AVG_DAILY_CONSUMPTION,AVG_DAILY_CONSUMPTION_bis
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",30,31,81.566667,78.935484
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",27,28,81.888889,78.964286
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",30,31,81.666667,79.032258
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",29,30,83.827586,81.033333
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",30,31,85.800000,83.032258
...,...,...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3028,"397,18",123,124,24.617886,24.419355
51428,0632KO,2020-01-01,2020-03-31,2163,"283,57",90,91,24.033333,23.769231
51429,0632KO,2020-04-01,2020-06-30,2160,"312,55",90,91,24.000000,23.736264
51430,0632KO,2020-07-01,2020-09-30,2230,"294,07",91,92,24.505495,24.239130


In [8]:
# 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.
df['NB_OF_INVOICES'] = df.groupby(['SITE_NAME'])['SITE_NAME'].transform('count')
df

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_IN_PERIOD,DAYS_IN_PERIOD_bis,AVG_DAILY_CONSUMPTION,AVG_DAILY_CONSUMPTION_bis,NB_OF_INVOICES
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",30,31,81.566667,78.935484,35.0
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",27,28,81.888889,78.964286,35.0
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",30,31,81.666667,79.032258,35.0
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",29,30,83.827586,81.033333,35.0
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",30,31,85.800000,83.032258,35.0
...,...,...,...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3028,"397,18",123,124,24.617886,24.419355,4.0
51428,0632KO,2020-01-01,2020-03-31,2163,"283,57",90,91,24.033333,23.769231,4.0
51429,0632KO,2020-04-01,2020-06-30,2160,"312,55",90,91,24.000000,23.736264,4.0
51430,0632KO,2020-07-01,2020-09-30,2230,"294,07",91,92,24.505495,24.239130,4.0


In [10]:
# 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.
coef_variation = df.groupby(["SITE_NAME"])["AVG_DAILY_CONSUMPTION"].std() / df.groupby(["SITE_NAME"])["AVG_DAILY_CONSUMPTION"].mean()
coef_variation = coef_variation.map('{:.3%}'.format)
coef_variation

SITE_NAME
0001BB      6.607%
0001BR      5.049%
0001KO     10.447%
0002BB      6.684%
0002BR      8.402%
            ...   
9513BB      0.377%
9515BB    117.872%
9516BB      1.813%
9517BB      0.056%
9518BB      0.312%
Name: AVG_DAILY_CONSUMPTION, Length: 2402, dtype: object

In [ ]:
# 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi.
df.to_csv(r'/Users/heloisevanrenterghem/Documents/THPprojets/factures_energie_clean.csv')
coef_variation.to_csv(r'/Users/heloisevanrenterghem/Documents/THPprojets/factures_energie_coefficient_variation.csv')